In [1]:
import os
from src.video_preprocessing.download_videos.youtube_download import preprocess_video
from src.video_preprocessing.scene_detection.scene_detect import detect_scenes
from src.video_preprocessing.download_videos.download_utils import (
    transcribe_audio_files,
    extract_and_store_audio,
    transcription_to_text,
    create_metadata
)
from src.ocr.pytesseract_image_to_text import extract_text_from_image
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from PIL import Image

import torch
from loguru import logger
import pickle

from src.clip.clip_model import CLIPEmbeddingsModel

ModuleNotFoundError: No module named 'image_utils'

# Download Pipeline
Download a video from a specific URL on YouTube, then run:
- Scene detection
- Keyframe detection

The resulting data will be stored under `/data/raw/<NAME>`

In [ ]:
# Define options and input for downloading a video from youtube

# INSERT video name here
name = "biology_chapter_3_3"
# INSERT video URL here
url = "https://youtu.be/DZSEErNZ1d4?si=f6YxKQ9rP6iqgTfk"
# INSERT chunk length in seconds 30s --> 30, no splitting: None
chunks = None

opts_aud = {"format": "mp3/bestaudio/best", "keep-video": True}
opts_vid = {"format": "mp4/bestvideo/best"}

In [4]:
# Downloads the video creates the relevant datafolders and transcribes the video
data_path = preprocess_video(
    download=True,
    uploaded_vid="ignore",  # path to local file
    url=url,
    name=name,
    aud_opts=opts_aud,
    vid_opts=opts_vid,  # Video download settings
    audio_file=name + ".mp3",
    input_file=name + ".mp4",
    output="output.mp4",
    split_length=chunks,
)

2024-06-17 10:31:09.105 | INFO     | src.video_preprocessing.download_videos.youtube_download:preprocess_video:49 - Starting AutoCaptioning...
2024-06-17 10:31:09.105 | INFO     | src.video_preprocessing.download_videos.youtube_download:preprocess_video:50 - Results will be stored in data/raw/biology_chapter_3_3
2024-06-17 10:31:09.105 | INFO     | src.video_preprocessing.download_videos.youtube_download:preprocess_video:58 - Created chunks folders


[youtube] Extracting URL: https://youtu.be/DZSEErNZ1d4?si=f6YxKQ9rP6iqgTfk
[youtube] DZSEErNZ1d4: Downloading webpage
[youtube] DZSEErNZ1d4: Downloading ios player API JSON
[youtube] DZSEErNZ1d4: Downloading m3u8 information
[info] DZSEErNZ1d4: Downloading 1 format(s): 22
[download] C:\Users\baatout\PycharmProjects\afm-vlm\data\raw\biology_chapter_3_3\biology_chapter_3_3.mp4 has already been downloaded
[download] 100% of  126.17MiB


2024-06-17 10:31:11.924 | INFO     | src.video_preprocessing.download_videos.youtube_download:preprocess_video:91 - Video is not splitted:
2024-06-17 10:31:11.924 | INFO     | src.video_preprocessing.download_videos.youtube_download:preprocess_video:95 - Video downloaded successfully!


In [5]:
#  Now that we have downloaded the video we want to perform scene_Detection:
detect_scenes(data_path)

2024-06-17 10:31:11.939 | INFO     | src.video_preprocessing.scene_detection.scene_detect:detect_scenes:29 - Found file
2024-06-17 10:31:11.939 | INFO     | src.video_preprocessing.scene_detection.scene_detect:detect_scenes:33 - Name:biology_chapter_3_3.mp4,dirname:C:\Users\baatout\PycharmProjects\afm-vlm\data/raw\biology_chapter_3_3\biology_chapter_3_3.mp4
2024-06-17 10:31:11.939 | INFO     | src.video_preprocessing.scene_detection.scene_detect:detect_scenes:35 - Running scene_detection:
2024-06-17 10:39:51.403 | INFO     | src.video_preprocessing.scene_detection.scene_detect:detect_scenes:29 - Found file
2024-06-17 10:39:51.403 | INFO     | src.video_preprocessing.scene_detection.scene_detect:detect_scenes:33 - Name:biology_chapter_3_3-Scene-001.mp4,dirname:C:\Users\baatout\PycharmProjects\afm-vlm\data/raw\biology_chapter_3_3\biology_chapter_3_3-Scene-001.mp4
2024-06-17 10:39:51.403 | INFO     | src.video_preprocessing.scene_detection.scene_detect:detect_scenes:35 - Running scene_det

In [6]:
# Extract the audio per detected scene
extract_and_store_audio(
    os.path.join(data_path, "scene_snippets"),
    os.path.join(data_path, "audio_chunks"),
)

  0%|          | 0/109 [00:00<?, ?it/s]

MoviePy - Writing audio in C:\Users\baatout\PycharmProjects\afm-vlm\data/raw\biology_chapter_3_3\audio_chunks\biology_chapter_3_3-Scene-001.wav



  1%|          | 1/109 [00:00<01:16,  1.42it/s]

MoviePy - Done.


  1%|          | 1/109 [00:01<01:16,  1.42it/s]

...

MoviePy - Done.


 99%|█████████▉| 108/109 [01:28<00:00,  1.11it/s]

MoviePy - Writing audio in C:\Users\baatout\PycharmProjects\afm-vlm\data/raw\biology_chapter_3_3\audio_chunks\biology_chapter_3_3-Scene-109.wav



100%|██████████| 109/109 [01:28<00:00,  1.23it/s]

MoviePy - Done.


# Audio Transcription using Whisper

For Faster Inference Please Use Tiny!

In [8]:
# Transcribe the different snippets:
audio_dir = os.path.join(data_path, "audio_chunks")
transcriptions_dir = os.path.join(data_path, "transcriptions")

model_type = "tiny"  # change to 'large' if you want more accurate results,
# change to 'medium.en' or 'large.en' for all english language tasks,
# and change to 'small' or 'base' for faster inference
lang = "en"

# Run whisper on all .wav files in audio_dir
transcribe_audio_files(audio_dir, transcriptions_dir, model_type=model_type, lang=lang)

2024-06-17 10:44:22.751 | INFO     | src.video_preprocessing.download_videos.download_utils:transcribe_audio_files:362 - Starting pooling:
  0%|          | 0/109 [00:05<?, ?it/s]


AttributeError: module 'whisper' has no attribute 'load_model'

# Load the CLIP Model

In [6]:
from src.clip.clip_model import CLIPEmbeddingsModel

# create instance
clip_model = CLIPEmbeddingsModel()

<Figure size 800x2000 with 0 Axes>

In [24]:
import tqdm
from pathlib import Path

# get current directory
# Get the path of the current notebook
notebook_path = Path().resolve()
image_path = os.path.join(notebook_path, 'data', 'raw', 'biology_chapter_3_3', 'extracted_keyframes')

images = []
# make a list out of the images
for image in os.listdir(image_path):
    if image.endswith(".jpg"):
        images.append(os.path.join(image_path, image))

# load and process the dataset
image_dataset = clip_model.load_and_process_dataset(images)

logger.info(f"Image_dataset: {image_dataset}")

2024-06-17 12:41:02.478 | INFO     | __main__:<module>:18 - Image_dataset: [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80D7FD00>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80827C70>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80826C80>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80827E80>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80825BD0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80826F50>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80826F80>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80826FB0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80826020>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80825C60>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=966x720 at 0x23F80825EA0>, <PIL.JpegImagePlugin.J

# Starting the Analysis of the Information Contained in the Video

### Inputs

* **Transcriptions**: [insert description or link to transcription]
* **Extraction from Slides using OCR**: [insert description or link to extracted content]
* **Textual Interpretation of Visual Information using LLAVA**: [insert description or link to 
interpreted information]

In [33]:
import json
import tqdm

# Transform transcription file

keyframes = {}
ocr_extracted_text = []

for filename in tqdm.tqdm(os.listdir(image_path)):
    # Check if the file ends with the specified extension
    if filename.endswith(".jpg"):
        filepath = os.path.join(image_path, filename)
        keyframe_num = int(filename.split("-")[2])

        # Extract text using OCR:
        ocr_text = extract_text_from_image(filepath)
        # TODO: DELETE THIS CODE - Crop the text to 50 characters
        ocr_extracted_text.append(ocr_text)
        logger.info(f"OCR_results: {ocr_extracted_text}")

        transcription_file_path = os.path.join(
            os.path.dirname(image_path),
            "transcriptions",
            filename.replace("-01.jpg", ".csv"),
        )

        transcription, timestamps = transcription_to_text(transcription_file_path)
        logger.info(f"Transcription_text: {transcription}")

        # Extract textual understanding of Visual features using LLAVA:

        # llava_results = generate_caption_using_llava(filepath)
        # logger.info(f"LLava_results: {llava_results}")

        # clip_llm_summary = prompt_llm_summary(
        #     slide_content=ocr_extracted_text,
        #     transcription=transcription,
        #     llava_output=llava_results,
        # )

        # extensive_summary = prompt_llm_extensive_summary(
        #     slide_content=ocr_extracted_text,
        #     transcription=transcription,
        #     llava_output=llava_results,
        # )

        # Alternative that goes faster.
        llava_results = "llava_results"
        clip_llm_summary = "clip_llm_summary"
        extensive_summary = "extensive_summary"
        
        # generate embeddings
        opened_image = Image.open(filepath)
        embeddings = clip_model.generate_image_embeddings(ocr_text, opened_image)
        clip_text_embedding = embeddings['text_embeds']
        clip_image_embedding = embeddings['image_embeds']

        keyframe, keyframe_metadata = create_metadata(
            keyframe_num,
            filepath,
            timestamps,
            transcription,
            ocr_extracted_text,
            llava_results,
            clip_llm_summary,
            extensive_summary,
            clip_text_embedding,
            clip_image_embedding,
        )
        keyframes[keyframe] = keyframe_metadata
        print(keyframes)

    # Save keyframes dictionary as Pickle

# Save with pickle
with open("data.pickle", "wb") as file:
    pickle.dump(keyframes, file)

  0%|          | 0/111 [00:00<?, ?it/s]2024-06-17 23:02:11.504 | INFO     | src.ocr.pytesseract_image_to_text:extract_text_from_image:17 - Extracted text: Chapter 3:

Nutrients & Membrane Transport
***Is It Possible to Supplement Your
Way to Better Performance & Health?

Biology 1020:
CURRENT TOPICS IN
BIOLOGY


2024-06-17 23:02:11.504 | INFO     | __main__:<module>:19 - OCR_results: ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n']
2024-06-17 23:02:11.504 | INFO     | __main__:<module>:28 - Transcription_text:  Welcome to Chapter 3, Nutrients, Membranes and Trin...
2024-06-17 23:02:11.834 | INFO     | src.clip.clip_model:process_clip_tensors:90 - Image embeddings shape: torch.Size([1, 512])
2024-06-17 23:02:11.834 | INFO     | src.clip.clip_model:process_clip_tensors:91 - Text embeddings shape: torch.Size([1, 512])
2024-06-17 23:02:11.863 | INFO     | src.clip.cli

{1: {'img_path': 'C:\\Users\\baatout\\PycharmProjects\\afm-vlm\\data\\raw\\biology_chapter_3_3\\extracted_keyframes\\biology_chapter_3_3-Scene-001-01.jpg', 'timestamps': [0, 5], 'transcription': ' Welcome to Chapter 3, Nutrients, Membranes and Trin...', 'ocr_extracted_text': ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n'], 'llava_result': 'llava_results', 'clip_text': 'clip_llm_summary', 'llm_long_summary': 'extensive_summary', 'clip_text_embedding': tensor([[-6.3477e-03,  4.1144e-02, -6.2738e-03, -1.5998e-02, -1.5231e-02,
          5.2521e-03,  2.1959e-03,  1.5892e-01,  6.9392e-02,  2.2310e-02,
         -1.1350e-02,  2.7823e-03,  1.8811e-02,  2.6283e-02, -2.5208e-02,
          5.3009e-02, -1.8662e-02,  3.6511e-02, -1.4450e-02,  6.5905e-04,
         -9.1668e-02,  4.0422e-02,  4.8536e-03, -1.6253e-02,  1.2104e-02,
         -6.4063e-03, -2.5818e-04,  7.8357e-03,  1

2024-06-17 23:02:12.419 | INFO     | src.ocr.pytesseract_image_to_text:extract_text_from_image:17 - Extracted text: Chapter 3:
Nutrients & Membrane Transport

***Is It Possible to Supplement Your
Way to Better Performance & Health?

Biology 1020:
CURRENT TOPICS IN
BIOLOGY


2024-06-17 23:02:12.419 | INFO     | __main__:<module>:19 - OCR_results: ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\nNutrients & Membrane Transport\n\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n']
2024-06-17 23:02:12.423 | INFO     | __main__:<module>:28 - Transcription_text:  transport. In this chapter we're going to look at those concepts but we're also going to be asking the question,  is it possible to supplement your way to better performance and health? We live in a world today  where suppleme

{1: {'img_path': 'C:\\Users\\baatout\\PycharmProjects\\afm-vlm\\data\\raw\\biology_chapter_3_3\\extracted_keyframes\\biology_chapter_3_3-Scene-001-01.jpg', 'timestamps': [0, 5], 'transcription': ' Welcome to Chapter 3, Nutrients, Membranes and Trin...', 'ocr_extracted_text': ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\nNutrients & Membrane Transport\n\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n'], 'llava_result': 'llava_results', 'clip_text': 'clip_llm_summary', 'llm_long_summary': 'extensive_summary', 'clip_text_embedding': tensor([[-6.3477e-03,  4.1144e-02, -6.2738e-03, -1.5998e-02, -1.5231e-02,
          5.2521e-03,  2.1959e-03,  1.5892e-01,  6.9392e-02,  2.2310e-02,
         -1.1350e-02,  2.7823e-03,  1.8811e-02,  2.6283e-02, -2.5208e-02,
          5.3009e-02, -1.8

2024-06-17 23:02:13.242 | INFO     | src.ocr.pytesseract_image_to_text:extract_text_from_image:17 - Extracted text: Chapter 3:

Nutrients & Membrane Transport
***Is It Possible to Supplement Your
Way to Better Performance & Health?

Biology 1020:
CURRENT TOPICS IN
BIOLOGY


2024-06-17 23:02:13.242 | INFO     | __main__:<module>:19 - OCR_results: ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\nNutrients & Membrane Transport\n\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n']
2024-06-17 23:02:13.242 | INFO     | __main__:<module>:28 - Transcription_text: 
2024-06-17 23:02:13.462 | INFO     | src.clip.clip_mod

{1: {'img_path': 'C:\\Users\\baatout\\PycharmProjects\\afm-vlm\\data\\raw\\biology_chapter_3_3\\extracted_keyframes\\biology_chapter_3_3-Scene-001-01.jpg', 'timestamps': [0, 5], 'transcription': ' Welcome to Chapter 3, Nutrients, Membranes and Trin...', 'ocr_extracted_text': ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\nNutrients & Membrane Transport\n\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n'], 'llava_result': 'llava_results', 'clip_text': 'clip_llm_summary', 'llm_long_summary': 'extensive_summary', 'clip_text_embedding': tensor([[-6.3477e-03,  4.1144e-02, -6.2738e-03, -1.5998e-02, -1.5231e-02,
  

2024-06-17 23:02:14.272 | INFO     | src.ocr.pytesseract_image_to_text:extract_text_from_image:17 - Extracted text: Learning Outcomes

Describe the role of nutrients in the body.
Describe the function of water in the body.

Describe the major dietary macronutrients and discuss the
functions of each.

List the major dietary micronutrients and describe their
functions.

Describe the structure and function of the plasma membrane.

Describe the structure and function of the subcellular
organelles.

Distinguish between passive transport and active transport.

Describe the processes of endocytosis and exocytosis.

2024-06-17 23:02:14.272 | INFO     | __main__:<module>:19 - OCR_results: ['Chapter 3:\n\nNutrients & Membrane Transport\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\nCURRENT TOPICS IN\nBIOLOGY\n\n', 'Chapter 3:\nNutrients & Membrane Transport\n\n***Is It Possible to Supplement Your\nWay to Better Performance & Health?\n\nBiology 1020:\

RuntimeError: The size of tensor a (93) must match the size of tensor b (77) at non-singleton dimension 1

In [ ]:
# Assuming keyframes is your dictionary
keyframes_count = len(keyframes)
print("Number of keyframes:", keyframes_count)

In [ ]:
def get_model_info(model_ID, device):
    # Save the model to device
    model = CLIPModel.from_pretrained(model_ID).to(device)
    # Get the processor
    processor = CLIPProcessor.from_pretrained(model_ID)
    # Get the tokenizer
    tokenizer = CLIPTokenizer.from_pretrained(model_ID)
    # Return model, processor & tokenizer
    return model, processor, tokenizer


# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
# Define the model ID
model_ID = "openai/clip-vit-base-patch32"
# Get model, processor & tokenizer
model, processor, tokenizer = get_model_info(model_ID, device)


def get_image(image_path):
    image = Image.open(image_path)
    # Convert the image to RGB
    rgb_image = image.convert("RGB")
    return rgb_image


def get_single_image_embedding(text, my_image, processor, model, device):
    image = processor(text=text, images=my_image, return_tensors="pt")[
        "pixel_values"
    ].to(device)
    embedding = model.get_image_features(image)
    # convert the embeddings to numpy array
    return embedding.cpu().detach().numpy()


one_image = get_image(
    image_path="/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3/extracted_keyframes/biology_chapter_3_3-Scene-055-01.jpg"
)

one_vector = get_single_image_embedding(
    response, one_image, processor, model, device
)  # Simple test

In [ ]:
# Generated one embedding?

print(one_vector)

# Load the CLIP Model

# Generate embeddings

# Generate Dataset Metadata

# Store Dataset Locally

# Test 1: Search for exact similar Text.

In [ ]:
test_image_path = os.path.join(base_dir, 'data', 'raw', 'biology_chapter_3_3', 'extracted_keyframes',
                               'biology_chapter_3_3-Scene-097-01.jpg')

test_text_description = extract_text_from_image(test_image_path)
# Search for similar images in database
clip_model.search_similar_images(test_text_description)

# Test 2: Search for for a slightly different Text.

In [ ]:
query_text = "plasma membrane and stuff going on"

clip_model.search_similar_images(query_text)